In [1]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
from src.models.ease.ease import EASE
from src.models.cb_iknn.cb_iknn import CBItemKNN
from src.models.hybrid_item_sim.hybrid_item_sim import HybridItemSimilarity
import pandas as pd
from src.evaluation import compute_mrr

In [2]:
dataset = Dataset()

In [3]:
split_dict = dataset.get_split()

In [4]:
train, train_label = split_dict[TRAIN]

In [5]:
val, val_label = split_dict[VAL]

In [6]:
iknn = ItemKNN(dataset, topk=1000, normalization=False, t1=0.3, t2=0, c=0.3, l=1, time_weight=None)
cb_iknn = CBItemKNN(dataset, topk=200)

In [7]:
hybrid_m = HybridItemSimilarity(dataset=dataset, model_list=[iknn, cb_iknn], model_weight_list=[0.7, 0.7],
                                normalization="l1", normalization_axis=1, time_weight=50)

[0.5, 0.5]


In [8]:
hybrid_m.compute_similarity_matrix(train)

Done: 100%|██████████| 23691/23691 [00:00<00:00, 27231.35it/s]


KeyboardInterrupt: 

In [ ]:
recs = hybrid_m.recommend(
    interactions=val,
    remove_seen=True,
    cutoff=100,
    leaderboard=False
)

In [ ]:
compute_mrr(recs, val_label)

## SUBMISSION

In [ ]:
full_data = dataset.get_train_sessions()
lead_data = dataset.get_test_leaderboard_sessions()

In [ ]:
hybrid_m.compute_similarity_matrix(full_data)
recs = hybrid_m.recommend(
    interactions=lead_data,
    remove_seen=True,
    cutoff=100,
    leaderboard=True
)
dataset.create_submission(recs, sub_name="hybrid_item_sim")